In [1]:
import os
# Specify the new directory path
parent_directory = 'D:/Projects/CornYield/src'

# Change the current directory
os.chdir(parent_directory) 
# %pwd


In [2]:
import numpy as np
from sklearn.model_selection import train_test_split

import torch
import torch.optim as optim

from dataset import select_data_and_yield_list
from ml.ml_predict import plot_result_separate
from dl.dl_dataset import CornDataset
from dl.model import ResNetRegression, ResNetFeatures, NNRegression
from dl.train import train_with_cross_validation,  validate_dual_models
from dl.dl_prediction import data_prepare

In [3]:
from dataset import get_filename_metadata
from dl.dl_dataset import MixedDataset
from dl.train import train_with_cross_validation, train, validate, data_transform

In [4]:
yield_file = 'D:/Corn_Yield/BL2022_Yld.csv'

img_path = 'D:/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220705_DOY186_extracted_filled'
out_path = '../output/DOY186/'

is_save_model = True

In [5]:
import random

In [23]:
key_word_list = ['Ref_filled.tif']
analyze_variety_id = 0
for keyword in key_word_list:

    img_list, yield_pf = get_filename_metadata(img_path, yield_file, keyword)

    
    indices = yield_pf.index[yield_pf['Variety_int'] == analyze_variety_id].tolist()
    img_list = [img_list[i] for i in indices]
    yield_pf = yield_pf[yield_pf['Variety_int'] == analyze_variety_id]
    yield_pf = yield_pf.reset_index(drop=True)
    yield_list = list(yield_pf['Yield_Bu_Ac'])
    
    print(len(img_list))
    print(len(yield_pf))
    
    random_numbers = [random.randint(0, len(img_list)) for _ in range(100)]
    for i in random_numbers:
        # print(yield_pf.iloc[i, 1])
        # print(img_list[i][77:89])
        print(img_list[i][77:89] == yield_pf.iloc[i, 1])

 

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


IndexError: list index out of range

In [ ]:
    
selection = ['Pioneer'] # 
# selection = 'Pioneer Deficit' 
# selection = 'Pioneer Full'

# key_word_list = ['Ref_filled.tif', 'RGB_filled.tif']
key_word_list = ['Ref_filled.tif']
suffix_list_list = [[], ['LWIR_filled.tif']]

# suffix = ['base', 'lwir']
# suffix_list = ['LWIR_filled.tif']
# VI_list = ['ndvi', 'ndre', 'gndvi', 'evi']
VI_list = ['evi']
for keyword in key_word_list:
    pioneer_yield_list, train_val_dataset, test_dataset, irrigate_type_list, test_indices= data_prepare(yield_file, img_path, out_path, keyword, selection)
    # yield_file = 'D:/Corn_Yield/BL2022_Yld.csv'
    doy_name = img_path[-23:-17]

            
    in_channel = 5
    num_epochs = 200
    batch_size = 32

    # Initialize an empty list to store fold-wise performance
    fold_accuracies = []

    # Initialize a new model for each fold
    # model = ResNetRegression(in_channel)
    model = ResNetFeatures(in_channel, 512)
    model2 = NNRegression(512)
    # model = EncoderCNN(in_channel, 1)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)
    model.to(device)
    model2.to(device)


    criterion = torch.nn.MSELoss()  # Mean Squared Error loss function
    # optimizer = optim.Adam(list(conv.parameters()) + list(deconv.parameters()), lr=0.001)  # Adam optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

    # optimizer.zero_grad()
    model = train_with_cross_validation(model, train_val_dataset, batch_size, num_epochs, optimizer, criterion, dual_model=True, model2=model2)

    if is_save_model:
        model_name = "path/model_" + model.__class__.__name__+ doy_name +"_Batch=" +str(batch_size) + "_state_dual.pth"
        torch.save(model.state_dict(), model_name)

    test_accuracy, test_prediction = validate_dual_models(model, model2, test_dataset, criterion, batch_size = batch_size, is_return_output = True)

    print(f'validation mse is {np.sqrt(np.mean(test_accuracy))}')


    name_tag = doy_name
    out_name = name_tag + ' ' + 'Pioneer' + ' ' 
    out_name = out_name + keyword[:-11] 
    out_name = out_name + ' CNN '

    yield_data = np.array(pioneer_yield_list)
    irrigate_data = np.array(irrigate_type_list)

    test_truth = yield_data[test_indices]
    plot_result_separate(np.array(test_truth), np.array(test_prediction), test_indices, irrigate_data, out_name)


In [ ]:
model2 = ResNetRegression(5,1)
print(model2)